<a href="https://colab.research.google.com/github/victorasso/wanderlust/blob/master/notebooks/041_lv_averaging_embeddings_generate_visualizations_tsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import glob
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/drive')
!rm -rf sample_data

Mounted at /content/drive


In [3]:
files = glob.glob('/content/drive/MyDrive/PISCHOOL/datasets/papers/source_asjc_codes/*')
print(len(files))

108


In [4]:
avg_embs = {}
for f in files:
  target = f.split('/')[-1].split('.')[0]
  
  emb = np.load(f)
  avg_embs[target] = np.mean(emb,axis=0)

In [5]:
avg_emb_df = pd.DataFrame.from_dict(avg_embs, orient='index')
avg_emb_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
3103,-0.646223,0.333282,0.029060,0.778521,0.117756,0.638356,0.571918,0.336703,0.885549,0.571962,-0.225186,0.318511,0.151262,-0.056868,-0.256253,0.078279,-1.062886,0.163467,-0.060480,-0.365955,-0.521911,-0.698267,-0.306263,-0.000011,0.568797,0.163279,0.219132,-0.139707,0.854415,-0.050394,-0.142940,1.162901,-0.197163,-0.251772,-0.130263,0.196188,-0.311952,0.987049,0.235560,0.655811,...,0.404582,0.076779,-0.889574,-0.325045,-0.757818,0.050200,0.756285,-0.543463,-0.253435,0.264062,0.205199,0.604703,-0.500276,0.243474,0.797202,-0.096544,0.138157,-0.338803,-0.536811,-0.325068,-0.000754,-0.055461,0.377982,0.049543,-1.338720,0.040659,0.476719,0.190703,-0.295432,0.251278,0.765890,-0.655602,0.888116,0.506910,-0.077671,0.546412,-0.224593,0.824599,0.140964,0.256999
3100,-0.428291,-0.051895,0.202590,0.398313,0.139053,0.558912,0.768021,0.278826,0.384540,0.713221,0.204483,0.296780,-0.090102,-0.032361,-0.375786,0.007824,-1.046480,0.480217,0.171768,-0.720474,-0.631145,-0.879199,-0.274947,0.076964,-0.340132,-0.062064,0.318119,0.077677,0.684866,0.201522,-0.154893,0.982732,0.096452,0.492029,-0.092497,0.555775,-0.473458,0.257672,0.153210,0.371787,...,0.519004,-0.046184,-0.622435,-0.566095,-0.514238,-0.031922,0.802402,-0.045361,-0.432677,0.165703,0.803519,0.600171,-0.078510,0.227013,0.633469,0.201072,0.413546,0.177977,-0.660133,-0.147569,-0.381484,0.074096,0.341325,-0.355869,-0.885972,0.028657,0.008868,0.104874,0.041966,0.416544,0.495869,-0.707472,0.854638,0.725033,-0.122831,0.335722,0.187199,0.664364,-0.046660,0.448285
2610,-0.295342,0.127636,0.007776,0.201588,0.358333,0.369516,0.796663,0.285075,0.393276,0.350529,-0.120791,0.289571,0.215830,0.116261,-0.278850,0.322606,-0.927778,0.384418,0.064312,-0.483599,-0.577830,-0.975757,-0.395033,0.481323,-0.068969,-0.043846,0.388206,0.148733,0.632857,0.301265,-0.122238,1.123544,-0.091453,0.276388,-0.167917,0.531806,-0.481522,0.422698,0.032704,0.302037,...,0.440851,0.001201,-0.725193,-0.318139,-0.412706,0.022760,0.963776,-0.063514,-0.379158,-0.032209,0.526185,0.478384,-0.310001,0.254452,0.649199,0.096235,0.388934,0.067187,-0.496918,-0.007246,-0.221115,-0.149368,0.337997,-0.234664,-1.108115,0.373898,-0.137137,0.030970,-0.180133,0.168841,0.469341,-0.708685,0.963790,0.616050,-0.093003,0.412253,0.254471,0.515069,0.092167,0.247839
3003,-0.286204,0.417999,0.119287,0.148014,-0.011930,-0.243778,0.418527,-0.077456,0.078549,0.414526,0.245209,0.154710,-0.313742,0.186162,-0.463535,0.010329,-1.093996,0.429150,0.151563,-0.681442,-0.884552,-0.287751,0.104645,1.020411,0.115406,-0.237082,0.399118,0.732322,0.462246,-0.211382,0.116739,0.758660,-0.093392,0.606437,-0.553365,0.488252,-0.383313,-0.120597,0.129229,0.273030,...,0.808075,0.020712,-0.502744,-0.741716,-0.093070,0.625583,0.127179,-0.247702,-0.576462,0.289752,0.496149,0.155632,-0.107390,0.516496,0.113161,0.144112,0.182587,0.199336,-0.781513,0.374858,-0.415642,-0.409313,0.624627,-0.346206,-0.315919,0.150962,0.338423,-0.547091,0.204782,0.864411,0.812514,-0.761161,0.856173,1.030119,0.128270,-0.043094,-0.010064,0.080316,0.255391,-0.292483
2739,-0.388265,0.036402,-0.095679,0.112262,-0.094862,0.599075,0.299708,0.438011,0.032446,1.070129,0.246554,0.100050,-0.135729,0.325876,-0.382196,-0.206975,-1.000965,-0.028957,0.124977,-0.601372,-0.607443,-0.492841,0.249299,0.378571,-0.565853,-0.187298,0.190732,-0.017781,0.257035,-0.156114,0.191099,0.576912,-0.149561,0.649866,-0.393533,0.271449,-0.398062,-0.005790,-0.064201,-0.150132,...,0.097332,0.037063,-0.195726,-0.498744,-0.328032,0.326902,0.703136,-0.241222,-0.961417,0.405856,0.513393,-0.006667,-0.270254,0.007961,0.510578,0.178392,0.058532,-0.064208,-0.973857,-0.445906,-0.553195,-0.102920,0.374213,-0.092261,-0.868398,0.093967,0.036480,0.018926,0.126651,0.417193,0.5482

In [6]:
similarity = np.zeros((len(avg_embs.keys()),len(avg_embs.keys())))

for i, emb_left in enumerate(avg_embs):
  for j, emb_right in enumerate(avg_embs):
    similarity[i,j] = cosine_similarity(avg_embs[emb_left].reshape(1,-1),avg_embs[emb_right].reshape(1,-1))

print(len(similarity))

108


In [10]:
similarity = {}

for i, emb_left in enumerate(avg_embs):
  sim = {}
  for j, emb_right in enumerate(avg_embs):
    sim[emb_right] = cosine_similarity(avg_embs[emb_left].reshape(1,-1),avg_embs[emb_right].reshape(1,-1))[0][0]

  similarity[emb_left] = sim

In [43]:
similarity_df = pd.DataFrame(similarity)
similarity_df

,3103,3100,2610,3003,2739,2730,3109,2302,3107,2310,3105,2311,2502,2611,2403,2307,2700,3110,2308,3002,2613,2215,2303,1706,3004,1607,3106,3005,2304,2312,1908,2506,2207,1507,2604,2705,2305,1603,1712,2213,...,1505,1602,2211,2002,1502,2504,3104,1605,2104,2500,1912,1503,2205,1408,2210,2508,2100,1300,1904,2105,1902,1600,1313,2103,1311,1900,1308,1700,1500,1312,1306,1110,1909,1304,1305,1307,1105,1111,1106,1104
3103,1.000000,0.905475,0.925674,0.782365,0.816942,0.783842,0.907843,0.760461,0.911155,0.813354,0.908493,0.805346,0.827354,0.887319,0.830980,0.810004,0.849408,0.832826,0.817652,0.781934,0.903904,0.791022,0.818692,0.884189,0.802725,0.856016,0.928455,0.791728,0.818515,0.805574,0.884602,0.813709,0.845646,0.823738,0.891534,0.798375,0.820656,0.805263,0.865369,0.849781,...,0.837345,0.800184,0.830828,0.811636,0.821127,0.865990,0.867902,0.822891,0.866981,0.846844,0.994020,0.799344,0.809028,0.832139,0.833390,0.838623,0.803376,0.871757,0.793868,0.814153,0.855131,0.838762,0.836900,0.820597,0.856500,0.847156,0.790225,0.864494,0.823837,0.858838,0.799875,0.780672,0.806974,0.867462,0.845333,0.848135,0.845342,0.777347,0.779439,0.814198
3100,0.905475,1.000000,0.962548,0.866102,0.917782,0.811842,0.959892,0.861304,0.978993,0.922961,0.976312,0.918345,0.945463,0.978503,0.863461,0.904616,0.937163,0.956158,0.908913,0.851067,0.961942,0.920997,0.884577,0.965784,0.863474,0.937472,0.978644,0.879731,0.926114,0.900910,0.927433,0.939736,0.957376,0.952648,0.971455,0.843900,0.938266,0.926919,0.936521,0.962998,...,0.944371,0.889563,0.963472,0.870171,0.936837,0.976018,0.988970,0.916134,0.966619,0.972511,0.926127,0.901986,0.934720,0.913948,0.967047,0.964775,0.892626,0.904077,0.863216,0.924570,0.909091,0.965216,0.890592,0.942188,0.881797,0.914414,0.851937,0.944049,0.959809,0.892660,0.825911,0.835079,0.927714,0.908738,0.931730,0.877885,0.873690,0.859989,0.877756,0.878484
2610,0.925674,0.962548,1.000000,0.849584,0.866795,0.823560,0.996241,0.804937,0.969462,0.858424,0.949844,0.852313,0.920184,0.964826,0.867085,0.855748,0.894664,0.923808,0.855633,0.844634,0.987397,0.853611,0.846636,0.951830,0.856629,0.919973,0.959077,0.839869,0.862932,0.846077,0.890886,0.896675,0.920459,0.894074,0.970007,0.828805,0.874011,0.889234,0.926959,0.911615,...,0.931070,0.857814,0.916601,0.845705,0.899996,0.947704,0.937787,0.893144,0.910267,0.918462,0.926961,0.873413,0.870401,0.881216,0.904056,0.924910,0.842995,0.912713,0.820586,0.858930,0.864779,0.900983,0.891163,0.866380,0.893632,0.871626,0.843953,0.921070,0.882905,0.903067,0.839722,0.815165,0.869558,0.922619,0.905859,0.891123,0.868565,0.814433,0.835902,0.838716
3003,0.782365,0.866102,0.849584,1.000000,0.880382,0.847734,0.842910,0.861994,0.855360,0.877357,0.876527,0.881453,0.912324,0.870200,0.884699,0.911878,0.905579,0.882971,0.811186,0.970753,0.862962,0.801191,0.836761,0.891002,0.985904,0.972421,0.858795,0.930745,0.901052,0.844578,0.823813,0.873859,0.850602,0.817861,0.870825,0.844370,0.866765,0.930937,0.848752,0.877953,...,0.911975,0.976000,0.854599,0.799762,0.923361,0.869028,0.868776,0.972506,0.858240,0.873092,0.800552,0.905280,0.810065,0.846673,0.836722,0.893034,0.784091,0.895613,0.800619,0.816184,0.792431,0.897788,0.970385,0.824808,0.891074,0.846719,0.987350,0.849057,0.864609,0.900924,0.859913,0.831680,0.852860,0.909041,0.938096,0.889160,0.854765,0.807016,0.925883,0.837076
2739,0.816942,0.917782,0.866795,0.880382,1.000000,0.817864,0.856740,0.897496,0.888045,0.949716,0.916639,0.946176,0.881140,0.919311,0.889583,0.965457,0.982332,0.874974,0.918613,0.848003,0.886103,0.924697,0.898849,0.932823,0.886098,0.915988,0.901115,0.944615,0.961187,0.906824,0.893842,0.869111,0.922438,0.883793,0.914067,0.866032,0.959294,0.878423,0.913677,0.954013,...,0.869848,0.912496,0.898043,0.887211,0.900186,0.885980,0.909592,0.899678,0.925200,0.913559,0.848680,0.847359,0.924617,0.926496,0.916287,0.888505,0.873240,0.894117,0.869427,0.892716,0.902505,0.937751,0.891546,0.901681,0.872828,0.921516,0.876741,0.920907,0.919326,0.878612,0.830842,0.832824,0.908047,0.872428,0.91

In [63]:
url = 'https://service.elsevier.com/app/answers/detail/a_id/15181/supporthub/scopus/~/what-is-the-complete-list-of-scopus-subject-areas-and-all-science-journal/'

asjc_df = pd.read_html(url)
asjc_df = dfs[0].astype(str).set_index('Code')
asjc_df = asjc_df.rename_axis(None)
meta_df = similarity_df.join(asjc_df).reset_index()[['index','Field','Subject area']]
meta_df

,index,Field,Subject area
0,3103,Astronomy and Astrophysics,Physical Sciences
1,3100,General Physics and Astronomy,Physical Sciences
2,2610,Mathematical Physics,Physical Sciences
3,3003,Pharmaceutical Science,Life Sciences
4,2739,"Public Health, Environmental and Occupational ...",Health Sciences
...,...,...,...
103,1307,Cell Biology,Life Sciences
104,1105,"Ecology, Evolution, Behavior and Systematics",Life Sciences
105,1111,Soil Science,Life Sciences
106,1106,Food Science,Life Sciences


In [67]:
# upload those in https://projector.tensorflow.org/

similarity_df.to_csv('output.tsv', sep='\t', index=None, header=None)
meta_df.to_csv('metadata.tsv', index=False, sep='\t')